In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# NOTE: this is now out of date!!!

from dynadojo.baselines.sindy import SINDy
from dynadojo.systems.lds import LDSystem
from dynadojo.systems.lorenz import LorenzSystem
from dynadojo.challenges import FixedComplexity

import pysindy as ps

In [4]:
timesteps = 10
# challenge = FixedComplexity(N=[1, 50, 500], l=3, e=None, t=timesteps, trials=1, control_horizons=0, max_control_cost_per_dim=0, test_examples=10, test_timesteps=50, system_cls=LorenzSystem, system_kwargs={"noise_scale": 0.01})
challenge = FixedComplexity(N=[20, 100, 1000], l=2, e=None,
    t=timesteps, trials=2, control_horizons=0, 
    max_control_cost_per_dim=0, test_examples=2, 
    test_timesteps=2, system_cls=LDSystem)

In [5]:
results = challenge.evaluate(SINDy,
    seed=100,
    noisy=True, 
    num_parallel_cpu = -1,
    id="sindy")
results.head()

,job_id,trial,latent_dim,embed_dim,timesteps,n,error,ood_error,total_cost,duration,system_seed,algo_seed,id
0,0,1,2,2,10,20,0.000020,None,0,0.047971,3293744024,533341188,sindy
1,0,1,2,2,10,100,0.000019,None,0,0.071537,3293744024,533341188,sindy
2,0,1,2,2,10,1000,0.000019,None,0,0.339700,3293744024,533341188,sindy
0,1,2,2,2,10,20,0.004057,None,0,0.038663,3586218795,2562180036,sindy
1,1,2,2,2,10,100,0.004089,None,0,0.059198,3586218795,2562180036,sindy


# Other Differentiation Methods

In [6]:
# Very sensitive to the smoothing parameters
smoothed_fd = challenge.evaluate(SINDy, 
    model_kwargs={"differentiation_method": ps.SmoothedFiniteDifference(
        smoother_kws={ 
            'window_length': timesteps//2,
            'polyorder': timesteps//4
        })}, 
    noisy=True, id="smoothed finite diff")
smoothed_fd.head()

TypeError: ScalingChallenge.evaluate() got an unexpected keyword argument 'model_kwargs'

In [70]:
fd = challenge.evaluate(SINDy, model_kwargs={"differentiation_method": ps.FiniteDifference()}, noisy=True, id="finite diff")
fd.head()

rep_id=0, latent_dim=2, embed_dim=2, n=5, t=50, control_h=0, total_cost=0, error=1.46e-05, ood_error=NA,model_seed=478491815, sys_seed=4285026285
rep_id=0, latent_dim=2, embed_dim=2, n=10, t=50, control_h=0, total_cost=0, error=1.8e-05, ood_error=NA,model_seed=478491815, sys_seed=4285026285
rep_id=0, latent_dim=2, embed_dim=2, n=50, t=50, control_h=0, total_cost=0, error=1.75e-05, ood_error=NA,model_seed=478491815, sys_seed=4285026285


,rep,latent_dim,embed_dim,timesteps,n,error,ood_error,total_cost,duration,system_seed,model_seed,id,control_horizon
0,0,2,2,50,5,0.000015,None,0,0.244657,4285026285,478491815,finite diff,0
1,0,2,2,50,10,0.000018,None,0,0.242584,4285026285,478491815,finite diff,0
2,0,2,2,50,50,0.000017,None,0,0.250348,4285026285,478491815,finite diff,0


In [71]:
tf = challenge.evaluate(SINDy, model_kwargs={"differentiation_method": ps.SINDyDerivative(kind='trend_filtered', order=0, alpha=1e-2)}, noisy=True, id="trend filtered")
tf.head()

rep_id=0, latent_dim=2, embed_dim=2, n=5, t=50, control_h=0, total_cost=0, error=1.52, ood_error=NA,model_seed=4289484824, sys_seed=2442242350
rep_id=0, latent_dim=2, embed_dim=2, n=10, t=50, control_h=0, total_cost=0, error=1.79, ood_error=NA,model_seed=4289484824, sys_seed=2442242350
rep_id=0, latent_dim=2, embed_dim=2, n=50, t=50, control_h=0, total_cost=0, error=0.0692, ood_error=NA,model_seed=4289484824, sys_seed=2442242350


,rep,latent_dim,embed_dim,timesteps,n,error,ood_error,total_cost,duration,system_seed,model_seed,id,control_horizon
0,0,2,2,50,5,1.518529,None,0,0.186346,2442242350,4289484824,trend filtered,0
1,0,2,2,50,10,1.787316,None,0,0.168418,2442242350,4289484824,trend filtered,0
2,0,2,2,50,50,0.069229,None,0,0.318911,2442242350,4289484824,trend filtered,0


In [72]:
spline = challenge.evaluate(SINDy, model_kwargs={"differentiation_method": ps.SINDyDerivative(kind='spline', s=1e-2)}, noisy=True, id="spline")
spline.head()

rep_id=0, latent_dim=2, embed_dim=2, n=5, t=50, control_h=0, total_cost=0, error=0.0554, ood_error=NA,model_seed=2001965979, sys_seed=2238852262
rep_id=0, latent_dim=2, embed_dim=2, n=10, t=50, control_h=0, total_cost=0, error=0.0492, ood_error=NA,model_seed=2001965979, sys_seed=2238852262
rep_id=0, latent_dim=2, embed_dim=2, n=50, t=50, control_h=0, total_cost=0, error=0.000511, ood_error=NA,model_seed=2001965979, sys_seed=2238852262


,rep,latent_dim,embed_dim,timesteps,n,error,ood_error,total_cost,duration,system_seed,model_seed,id,control_horizon
0,0,2,2,50,5,0.055367,None,0,0.310694,2238852262,2001965979,spline,0
1,0,2,2,50,10,0.049223,None,0,0.304703,2238852262,2001965979,spline,0
2,0,2,2,50,50,0.000511,None,0,0.282101,2238852262,2001965979,spline,0
